In [1]:
import pyodbc
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd

server = os.getenv("MSQL_Server")  
database = 'SAS'
username = os.getenv("USERNAME")
password = os.getenv("PWD")

conn_str = (
    f'DRIVER={{SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'Trusted_Connection=yes;')
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

cursor.execute("SELECT * FROM [SAS].[dbo].[F564205A]") # tabelle top_products
rows = cursor.fetchall()

df = pd.DataFrame.from_records(rows, columns=[desc[0] for desc in cursor.description])

print(df)
cursor.close()
conn.close()


OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server existiert nicht oder Zugriff verweigert. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (67)')

In [4]:
import pyodbc
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd

server = os.getenv("MSQL_Server")  
database = 'SAS'
username = os.getenv("USERNAME")
password = os.getenv("PWD")

conn_str = (
    f'DRIVER={{SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'Trusted_Connection=yes;')
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

cursor.execute("SELECT * FROM [SAS].[dbo].[top_products]")
rows = cursor.fetchall()

df = pd.DataFrame.from_records(rows, columns=[desc[0] for desc in cursor.description])

print(df)
cursor.close()
conn.close()


OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server existiert nicht oder Zugriff verweigert. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (67)')

In [2]:
import pyodbc

dsn = 'SAS'
user = 'SAS'
password = 'welcome'
server = 'JDEEXT01.MEDISERVICE.CH'
driver = 'Oracle in OraClient12Home1'

conn_str = f'DSN={dsn};UID={user};PWD={password};Server={server},Driver = {driver}'
conn = pyodbc.connect(conn_str)
# print drivers 
print(pyodbc.drivers())

cursor = conn.cursor()
cursor.execute("SELECT * FROM PROXY_JDE11_PRODDTA.ATC_CODES")
for row in cursor.fetchall():
    print(row)

cursor.close()
conn.close()


Error: ('IM014', '[IM014] [Microsoft][ODBC Driver Manager] Der angegebene DSN weist eine nicht übereinstimmende Architektur von Treiber und Anwendung auf. (0) (SQLDriverConnect)')

In [4]:
import platform
print(platform.architecture())
import pyodbc
print(pyodbc.drivers())


('64bit', 'WindowsPE')
['SQL Server', 'Oracle in OraClient12Home1']


In [6]:
import oracledb  # oder import cx_Oracle

# Verbindet sich über den TNS-Eintrag
connection = oracledb.connect(
    user = 'SAS',
    password = 'welcome',
    dsn = "JDE11.MEDISERVICE.CH"  # wie in tnsnames.ora definiert
)


DatabaseError: DPY-4027: no configuration directory specified

In [12]:
from agents import Agent, Runner
from backend.app.tools.compendium_api_tool_openai import compendium_api_tool
from agents import Agent, Runner
from agents import Agent, WebSearchTool

compendium_web_agent = Agent(
    name="CompendiumWebAgent",
    instructions=(
        "Beantworte Fragen ausschließlich mit Informationen von Compendium.ch. "
        "Wenn du keine passenden Informationen findest, sage das bitte deutlich."
    ),
    model="gpt-4o",
    tools=[WebSearchTool()],
)
combined_agent = Agent(
    name="PharmaAgent",
    instructions=(
        "Beantworte pharmazeutische Fragen mit Compendium.ch Webdaten. "
        "Wenn dort keine Information verfügbar ist, nutze lokale API-Daten."
    ),
    model="gpt-4o",
    tools=[
        compendium_web_agent.as_tool(
            tool_name="compendium_web_search",
            tool_description="Durchsucht Compendium.ch über Webzugriff"
        ),
        compendium_api_tool,
    ],
)

import asyncio

async def run_agent():
    urls = []
    result = await Runner.run(combined_agent, input="Welche Nebenwirkungen hat Dafalgan?")
    print(result.final_output)
    for item in result.output_items:
        if hasattr(item, "annotations"):
            for ann in item.annotations:
                if ann.type == "url_citation" and "compendium.ch" in ann.url:
                    urls.append(ann.url)
    if urls: 
        print("Gefundene URLs:")
        for url in urls:
            print(url)


asyncio.run(run_agent())



Dafalgan, das Paracetamol enthält, kann verschiedene Nebenwirkungen haben. Zu den häufigen gehört Erbrechen. Selten können Hautrötungen, allergische Reaktionen, Atembeschwerden, Veränderungen des Blutbildes und bestimmte Formen der Blutarmut auftreten. Sehr selten sind schwere Hauterkrankungen möglich.

Bei unkontrollierter Einnahme kann es zu einer Überdosierung kommen, die ernsthafte Leberschäden verursachen kann. Symptome sind Übelkeit, Erbrechen und Bauchschmerzen. Bei Auftreten von Nebenwirkungen oder vermuteter Überdosierung sollte umgehend ein Arzt konsultiert werden.

Für weitere Details und spezifische medizinische Beratung sollten Sie Ihren Arzt oder Apotheker kontaktieren.


AttributeError: 'RunResult' object has no attribute 'output_items'

In [17]:
import nest_asyncio
import asyncio
nest_asyncio.apply()

from agents import Agent, WebSearchTool, Runner
from IPython.display import display, Markdown

# Define the agent
compendium_web_agent = Agent(
    name="CompendiumWebAgent",
    model="gpt-4o",
    instructions=(
        "Beantworte medizinische Fragen ausschließlich basierend auf Informationen von Compendium.ch. "
        "Wenn keine zuverlässige Information auf Compendium.ch gefunden wird, sage dies explizit."
    ),
    tools=[WebSearchTool()],
)

# The query
query = "Was ist standarddosierung von Dafalgan Tabl 500mg bei einem körpergewicht von 40kg?"

# Async wrapper (Jupyter-safe)
async def run_in_jupyter():
    result = await Runner.run(compendium_web_agent, input=query)
    output = result.final_output

    # Gather compendium.ch links from annotations if present
    links = set()

    # Check final_response annotations
    if hasattr(result, "final_response") and result.final_response and hasattr(result.final_response, "annotations"):
        for ann in result.final_response.annotations:
            if getattr(ann, "type", "") == "url_citation" and "compendium.ch" in ann.url:
                links.add(ann.url)

    # Fallback: search for compendium.ch in the output itself
    import re
    found_links = re.findall(r"https?://(?:www\.)?compendium\.ch[^\s\)]*", output)
    links.update(found_links)

    return output, sorted(links)

# Run inside existing event loop
output_text, links = await run_in_jupyter()

# Display improved output
display(Markdown("## 💬 Antwort von Compendium.ch"))

# Format answer with paragraph spacing
cleaned_output = "\n\n".join(line.strip() for line in output_text.strip().split("\n") if line)
display(Markdown(cleaned_output))

# Show sources
if links:
    display(Markdown("\n---\n\n## 🔗 Quellen:\n"))
    for url in links:
        display(Markdown(f"- [compendium.ch]({url})"))
else:
    display(Markdown("\n---\n\n⚠️ **Hinweis:** Es wurden keine direkten Quellen von [compendium.ch](https://compendium.ch) gefunden."))



## 💬 Antwort von Compendium.ch

Gemäß den Informationen auf Compendium.ch beträgt die empfohlene Dosierung von Dafalgan 500 mg Tabletten für Kinder mit einem Körpergewicht von 30–40 kg (9–12 Jahre) eine Einzeldosis von 1 Tablette, mit einem maximalen Tageslimit von 4 Tabletten. ([compendium.ch](https://compendium.ch/om/product/1552047-dafalgan-tabl-500-mg-neu?utm_source=openai))

Für Kinder mit einem Körpergewicht von 22–30 kg (6–9 Jahre) wird eine Einzeldosis von ½–1 Tablette empfohlen, mit einer maximalen Tagesdosis von 3 Tabletten. ([compendium.ch](https://compendium.ch/om/product/1552047-dafalgan-tabl-500-mg-neu?utm_source=openai))

Da ein Körpergewicht von 40 kg an der oberen Grenze der 30–40 kg Kategorie liegt, ist die empfohlene Dosierung:

- **Einzeldosis:** 1 Tablette (500 mg)

- **Maximale Tagesdosis:** 4 Tabletten (entspricht 2 g Paracetamol)

Zwischen den Einzeldosen sollte ein Abstand von 6–8 Stunden eingehalten werden. ([compendium.ch](https://compendium.ch/om/product/1552047-dafalgan-tabl-500-mg-neu?utm_source=openai))

Bitte beachten Sie, dass die maximale kontinuierliche Anwendungsdauer ohne ärztliche Konsultation für Kinder bis 12 Jahre 3 Tage beträgt. ([compendium.ch](https://compendium.ch/om/product/1552047-dafalgan-tabl-500-mg-neu?utm_source=openai))

Es ist wichtig, die angegebene Dosierung nicht zu überschreiten und bei Unsicherheiten oder anhaltenden Beschwerden einen Arzt oder Apotheker zu konsultieren.


---

## 🔗 Quellen:


- [compendium.ch](https://compendium.ch/om/product/1552047-dafalgan-tabl-500-mg-neu?utm_source=openai)